In [1]:
from create_vectorDB import VECTORDB
from create_documents import DOCUMENTS

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [2]:
vec_obj = VECTORDB()
doc_obj = DOCUMENTS()
vec_dir = vec_obj.db_dir

# Read the Ollama embeddings
embeddings = OllamaEmbeddings(model="llama3.1", show_progress=False)
db = Chroma(persist_directory=vec_dir,
            embedding_function=embeddings)

# Create retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs= {"k": 5}
)

c:\Users\vinay\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [3]:
# Create Ollama language model - Llama3.1
local_llm = 'llama3.1'

llm = ChatOllama(model=local_llm,
                 keep_alive="3h", 
                 max_tokens=2048,  
                 temperature=0)

In [4]:
# Create prompt template
template = """<bos><start_of_turn>user\nYou are a chatbot built by ctruh. \
You will interact with uses who will have two types of messages. \
1. Feedback - user will give feedback, respond back appropriately and be empathetic. \
2. queries - user will ask some queries, respond back appropriately and be empathetic. \
Always respond enthusiastically and in a friendly manner to the user's messages. \
If there is an negative feedback, take the feedback and let them know that we'll look into it. \
Write in full sentences with correct spelling and punctuation. \
Do not give any additional information on the context, only stick to responding back to the user's message. \
If the context doesn't contain the answer, respond back saying that you are unable to find the answer. \
Format the answers appropriately with right new-lines and tabs and numbers wherever necessary. \
The context also has a sample set of feedback and the corresponding responses. \
Use it ONLY as a reference to help you understand how to respond back to the user feedbacks. \

CONTEXT: {context}

QUESTION: {question}

<end_of_turn>
<start_of_turn>model\n
ANSWER:"""
prompt = ChatPromptTemplate.from_template(template)

In [5]:
# Create the RAG chain using LCEL with prompt printing and streaming output
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [6]:
# Function to ask questions
def ask_question(question):
    print("User:", question)
    print("Cbot:", end=" ", flush=True)
    for chunk in rag_chain.stream(question):
        print(chunk.content, end="", flush=True)
    print("\n\n")

In [7]:
while True:
    user_question = input("Ask a question (or type 'quit' to exit): ")
    if user_question.lower() == 'quit':
        break
    answer = ask_question(user_question)
    # print("\nFull answer received.\n")

User: hello
Cbot: Hello! It's great to meet you. How can I assist you today?


User: the web page is really slow, and i am not able to create my assets because of that. You need to fix this
Cbot: We apologize for the slow website performance! Our team is constantly working on improving load times. Can you check your internet connectivity is reliable? We'll also look into optimizing our servers to ensure a smoother experience for all users.

Please let us know if there's anything else we can do to help. We're here to assist you in creating your assets!

Would you like to provide more feedback or ask another question?


User: i know you will not fix the issue
Cbot: I understand that you're feeling frustrated and don't think we can resolve the issue. I want to assure you that we take all feedback seriously, even if it's about something we might not be able to fix.

Can you please tell me more about what's not working for you? What specifically are you experiencing that's causing you to fe